# **Converting raw results into a NN-input**

In [1]:
import numpy as np
import pandas as pd

In [2]:
RAW_OCCUPATION = "../02_data/occupancies.csv"

In [3]:
raw_df = pd.read_csv(RAW_OCCUPATION)

In [20]:
raw_df.head()

,ids,timestamp,counts
0,1059,14460,0
1,1060,14460,0
2,1061,14460,0
3,1062,14460,0
4,1063,14460,0


## ID 1-hot encoding

In [14]:
id_list = list(raw_df.ids.unique())

In [15]:
id_encoding = np.zeros((len(raw_df), len(id_list)))

In [19]:
for i,r in raw_df.iterrows():
    id_encoding[i, id_list.index(r.ids)] = 1.0

## Adding timestamps and labels

In [33]:
timestamps = np.array(raw_df["timestamp"]).reshape(len(raw_df), 1)
timestamps = timestamps - min(timestamps)
timestamps = timestamps / max(timestamps)

In [34]:
counts = np.array(raw_df["counts"]).reshape(len(raw_df), 1)
counts = counts-np.mean(counts)
counts = counts / np.std(counts)

In [35]:
collected_df = np.hstack([id_encoding, timestamps, counts])

In [36]:
collected_df.shape

(1981200, 129)

## Saving result

In [38]:
pd.DataFrame(collected_df).to_csv("../02_data/nn.csv", header=None, index=None)